In [61]:
# library loads and function definitions

import numpy as np
import random as rng

# generate_nodes: input n -> output list like [0,1,...,n], up to 9. (BIG issue, need 
# to be able to simulate more nodes, should switch to tuples.
def generate_nodes(n):
    return list(range(0,n))

# generate_edged: input: list of nodes [0,1,...,n] -> output: list of 2-character strings 
# ex. '13' which designates edge from node 1 to node 3. Also generates "mirrored" edges like
# '31' to refer to the same edge as earlier, but with swapped starting and ending nodes.
def generate_edges(nodes):
    edges = []
    for i in nodes:
        for j in nodes:
            if i < j:
                edges.append(str(i)+str(j))
                edges.append(str(j)+str(i))
    return edges

# generate_distances: input a list of edges (both directions pairwise, "01" followed by 
# "10" or vice versa) and integers min_dist and max_dist -> 
# output dictionary with inputs as keys, and random values uniformly generated between 
# min_dist and max_dist assigned to each key, then assign the same value to each key where
# the string is reversed
def generate_distances(trimmed_edges, min_dist, max_dist):
    graph = {}
    for i in trimmed_edges:
        graph[i] = rng.uniform(min_dist,max_dist)
        graph[i[::-1]] = graph[i]
    
    return graph

# generate_probabilities: input: list of edges, distances dictionary, single path/edge, 
# float pheremone, and dictionary of in-going probabilities -> output: dict of updated probabilities
# (can be cleaned up since this function only get's run once for init_graph, update_graph 
# takes care of updating probabilities now)
def generate_probabilities(edges, distances ={}, path='', pheremone=0, in_probabilities = {}):
    out_probabilities = {}
    if in_probabilities == {}:
        for i in edges:
            out_probabilities[i] = 1
    
    else:
        out_probabilities = in_probabilities
        probabilities[path] += pheremone/distances[path]
    
    return out_probabilities

# assign_labels: input: list of nodes -> output dictionary with keys "colony" and "food"
# mapped to random integers from the inputted list of nodes.
def assign_labels(nodes):
    places = {}
    places["colony"] = nodes[rng.randint(0,nodes[-1])]
    places["food"] = nodes[rng.randint(0,nodes[-1])]
    
    while places["food"] == places["colony"]:
        places["food"] = nodes[rng.randint(0,nodes[-1])]
        
    return places

# move_ant: input: integer representing current postion node, graph dictionary the ant is
# walking on -> output: path string where first character is the curr_pos, and second is
# randomly chosen via probabilities in graph. Example: curr_pos = 1, move_ant(curr_pos, graph)
# will return '1x', where x is an integer not equal to 1.
def move_ant(curr_pos, graph):
    neighbors = dict(filter(lambda edge: edge[0][0] == str(curr_pos), graph.items()))
    path = rng.choices(list(neighbors.keys()), (list(i[0] \
                                                    for i in list(neighbors.values()))))[0]
    
    return path

# init_graph: input: number of nodes, float minimum distance between nodes, float maximum
# distance between nodes -> output: graph dictionary, where keys are all the paths/edges
# (like '01', '25', etc.) and the values are lists [distance, probability, pheremone],
# all floats. The probabilities are more accurately called weights, they can be any size,
# the update_graph function will normalize the choices available and scale it to a 
# probability space.
def init_graph(n, min_dist, max_dist):
    nodes = generate_nodes(n)
    places = assign_labels(nodes)
    edges = generate_edges(nodes)
    distances = generate_distances(edges, min_dist, max_dist)
    probabilities = generate_probabilities(edges)
    
    graph = {}
    pheremone = 0
    
    for i in edges:
        graph[i] = [distances[i],probabilities[i],pheremone]
        
    for i in places:
        graph[i] = places[i]

    return graph

# update_graph: input: graph dictionary, path string, pheremone float, evaporation rate float
# -> output: graph dictionary
# This will update a graph based on the path the ant just took. The idea is to take the
# result of move_ant and plug it into update_graph. The new ant_pos is set to the second
# character in the path returned by move_ant
def update_graph(graph, path, pheremone, evaporation_rate):
    graph[path][2] += pheremone
    graph[path][1] += pheremone/graph[path][0]
    graph[path[::-1]][1] = graph[path][1]
    not_path = dict(filter(lambda edge: edge[0] != path and \
                           edge[0] != path[::-1] and \
                           len(edge[0]) == 2, \
                           graph.items()))
    
    for i in list(not_path.keys()):
        graph[i][2] *= graph[i][2]*(1-evaporation_rate)
        graph[i][1] += graph[i][2]/graph[i][0]

    
    return graph


In [72]:
graph = init_graph(9,0,50) # input nodes, minimum distance, and maximum distance
K = 1 # pheremone amount put on path after each movement
V = 0.3 # evaporation rate, float in [0,1] to represent proportion of pheremone lost
food_needed = 5 # how many times we want the ant to get food and come back to the colony

In [73]:
ant_pos = graph["colony"]

food_amount = 0

while (food_amount < food_needed):
    has_food = False
    while (has_food == False):
        path = move_ant(ant_pos,graph)
        graph = update_graph(graph,path,K,V)
        ant_pos = int(path[1])
        if (ant_pos == graph["food"]):
            has_food = True
    
    while (has_food == True):
        path = move_ant(ant_pos,graph)
        graph = update_graph(graph,path,K,V)
        ant_pos = int(path[1])
        if (ant_pos == graph["colony"]):
            has_food = False
            food_amount += 1
            print("food aquired")

print(graph)
    

food aquired
food aquired
food aquired
food aquired
food aquired
{'01': [5.469669246113757, 1.0, 0.0], '10': [5.469669246113757, 1.0, 0.0], '02': [23.679946821157227, 1.0899544942102322, 0.0], '20': [23.679946821157227, 1.0422298245664359, 0.0], '03': [32.56046971080044, 1.1268445349807845, 0.0], '30': [32.56046971080044, 1.1268445349807845, 1.0], '04': [32.752281536109415, 1.126101677364642, 0.0], '40': [32.752281536109415, 1.0915966723323531, 0.0], '05': [9.981373222802075, 1.0, 0.0], '50': [9.981373222802075, 1.0, 0.0], '06': [22.179433485318782, 1.0, 0.0], '60': [22.179433485318782, 1.0, 0.0], '07': [11.268392601820775, 1.0, 0.0], '70': [11.268392601820775, 1.0, 0.0], '08': [14.540434146814507, 1.2840436260376187, 0.0], '80': [14.540434146814507, 1.3617660398123121, 0.0], '12': [40.196682825442714, 1.105866246667719, 0.08235429999999995], '21': [40.196682825442714, 1.0778700484518877, 0.0], '13': [14.491434383413566, 1.069006281472355, 0.0], '31': [14.491434383413566, 1.14699149738